### Model Architechture

Since the input consists of two parts, an **image vector** and a **partial caption**, we cannot use the Sequential API provided by the Keras library. 

For this reason, we use the Functional API 

In [1]:
from keras.models import Model,Input
from keras.applications.inception_v3 import InceptionV3,preprocess_input
from keras.layers import Embedding,Dense,BatchNormalization,Dropout,LSTM,add
from keras.utils import plot_model
from keras.utils import plot_model

Using TensorFlow backend.


In [2]:
# standard variables
MAX_LENGTH = 39
VOCAB_SIZE = 1665
NPIX = 299
TARGET_SIZE = (NPIX,NPIX,3)
EMBEDDING_SIZE = 300


In [3]:
# partial caption sequence model

inputs2 = Input(shape=(MAX_LENGTH,))
se1 = Embedding(VOCAB_SIZE, EMBEDDING_SIZE, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(300)(se2)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [4]:
# image feature extractor model
inputs1 = Input(shape=(2048,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(300, activation='relu')(fe1)

In [5]:
decoder1 = add([fe2, se3])
decoder2 = Dense(300, activation='relu')(decoder1)
outputs = Dense(VOCAB_SIZE, activation='softmax')(decoder2)

In [6]:
# merge the two input models
# image_feature + partial caption ===> output
model = Model(inputs=[inputs1, inputs2], outputs=outputs)

In [7]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 39)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 2048)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 39, 300)      499500      input_1[0][0]                    
__________________________________________________________________________________________________
dropout_2 (Dropout)             (None, 2048)         0           input_2[0][0]                    
__________________________________________________________________________________________________
dropout_1 

In [10]:
# setting wight of embedded matrix that we saved earlier for words
from pickle import load,dump
with open("embedding_matrix.pkl","rb") as f:
    embedding_matrix = load(f)

In [11]:
print(model.layers[2])

In [12]:
model.layers[2].set_weights([embedding_matrix])
model.layers[2].trainable = False

In [13]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [14]:
# keras library import  for Saving and loading model and weights

from keras.models import model_from_json
from keras.models import load_model

model_json = model.to_json()


with open("model.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights("model.h5")